In [ ]:
import json
import os
import javalang

test_methods = []

# Step 1: Identify the test files
app_dir = os.path.dirname(os.path.dirname(os.getcwd()))
app_dir = os.path.join(app_dir, "app", "ctest-flink-kubernetes", "flink-kubernetes", "src", "test")
for root, dirs, files in os.walk(app_dir):
    for file in files:
        if file.endswith("Test.java") or file.endswith("Tests.java"):
            with open(os.path.join(root, file), "r") as f:
                # Step 2: Extract test methods
                tree = javalang.parse.parse(f.read())
                rel_path = os.path.relpath(os.path.join(root, file), app_dir)
                rel_path = rel_path.replace(".java", "")
                rel_path = ".".join(rel_path.split(os.sep)[1:])
                for path, node in tree.filter(javalang.tree.MethodDeclaration):
                    for annotation in node.annotations:
                        if annotation.name == "Test":
                            # Step 3: Generate JSON
                            test_methods.append(f"{rel_path}#{node.name}")

# Step 4: Write to file
with open("test_method_list.json", "w") as f:
    f.write(json.dumps(test_methods))